In [1]:
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import platform
import matplotlib.pyplot as plt
%matplotlib inline

# This patch of code just ensures we get an easy to read font size for the duration of the notebook
import matplotlib
font = {'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

In [2]:
# Set file location based on platform. 
platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"

In [3]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_file = os.path.join(data_root,'allen-brain-observatory/visual-coding-2p/manifest.json')

# Create data cache object 
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [4]:
# Load in data for chosen container ID: 688678764

In [5]:
# Select the relevant data for chosen container ID
desired_container_id = 688678764
desired_container = boc.get_ophys_experiments(experiment_container_ids=[desired_container_id])
desired_container = sorted(desired_container, key=lambda x: x['session_type']) # sort based on session type so A comes first

# Get session IDs for each session
session_one_id = desired_container[0]["id"]

# load in session data for each session
session_one = boc.get_ophys_experiment_data(ophys_experiment_id=session_one_id)

# Get cells for each session
cells_one = session_one.get_cell_specimen_ids()

In [6]:
# Start with session A, drifting gratings

In [7]:
# here, my goal is calculate the mean neuron response to each stimulus, 
# containing both orientation and temporal frequency 

In [8]:
# Get events for session - all events at 1s interval timestamps
events = boc.get_ophys_experiment_events(ophys_experiment_id=session_one_id)

In [9]:
# Get stimulus table for each drifting gratings presentation
stim_table = session_one.get_stimulus_table("drifting_gratings")

# There are three presentations of drifting gratings per session

In [10]:
# Create arrays for orientation, temporal frequency, and response

In [34]:
# orientation matrix shows the orientation value for each frame, likewise for temporal frequency matrix
orientation = np.zeros(len(stim_table))
temp_freq = np.zeros(len(stim_table))
response = np.empty((len(cells_one), len(stim_table), 60)) # 3d array for responses at each frame to each stimulus for each neuron

# Loop through each stimulus presentation, store its parameters
for i in range(len(stim_table)):
    orientation[i] = stim_table.orientation[i]
    temp_freq[i] = stim_table.temporal_frequency[i]

for i in range(len(stim_table)):
    for cell_index in range(len(cells_one)):
        response[cell_index,i,:] = events[cell_index, stim_table.start[i]:stim_table.start[i]+60]
        
# response matrix contains the response at each frame for every stimulus presentation for each
# cell contained in the session

In [13]:
divide_indices = []

for i in range(len(stim_table)):
    try:
        if stim_table.start[i+1]-stim_table.end[i] > 100:
            divide_indices.append(i+1) # saves index where new stim presentation begins
    except:
        pass

In [68]:
response[2][138]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.10949111,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [14]:
divide_indices

[200, 400]

In [15]:
from event_utils import get_events

In [24]:
all_events = get_events(boc, session_one_id, "VISp")

In [25]:
all_events

,0,1,2,3,4,5,6,7,8,9,...,115758,115759,115760,115761,115762,115763,115764,115765,115766,115767
cell_id,,,,,,,,,,,,,,,,,,,,,
701728881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
701727459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
701728764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.152516,0.0,0.000000,0.0,0.0,0.0,0.0
701727473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
701733802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.270554,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701722576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
701722499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.133518,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
701722406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [62]:
np.where(all_events==max(all_events[2]))

(array([295]), array([2]))

In [26]:
all_events.iloc[2,115761]

0.15251641499924218

In [28]:
all_events.index

Index([701728881, 701727459, 701728764, 701727473, 701733802, 701728841,
       701723075, 701721074, 701728857, 701724624,
       ...
       701728404, 701722131, 701722167, 701721326, 701722523, 701722576,
       701722499, 701722406, 701732639, 701720025],
      dtype='int64', name='cell_id', length=475)